In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
'''os.chdir('/content')
!mkdir /root/.kaggle
!echo '{"username":"daniel1998xx","key":"c99226b7cd73b12d4681a78a5b02d227"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d shuyangli94/interview-npr-media-dialog-transcripts
if not os.path.exists("/content/NPR_Data"):
    os.makedirs("/content/NPR_Data")
'''

'os.chdir(\'/content\')\n!mkdir /root/.kaggle\n!echo \'{"username":"daniel1998xx","key":"c99226b7cd73b12d4681a78a5b02d227"}\' > /root/.kaggle/kaggle.json\n!kaggle datasets download -d shuyangli94/interview-npr-media-dialog-transcripts\nif not os.path.exists("/content/NPR_Data"):\n    os.makedirs("/content/NPR_Data")\n'

In [3]:
'''!unzip interview-npr-media-dialog-transcripts.zip -d NPR_Data'''

'!unzip interview-npr-media-dialog-transcripts.zip -d NPR_Data'

In [4]:
'''os.chdir('/content/NPR_Data')'''

"os.chdir('/content/NPR_Data')"

In [5]:
df = pd.read_csv('utterances.csv')
df['speaker'] = df['speaker'].str.upper()
df['speaker'] = df['speaker'].str.split(', H', expand=True)[0]

In [6]:
df = df[df['speaker'] != '_NO_SPEAKER']

In [7]:
df['utterance_len'] = df['utterance'].str.len()

In [8]:
df = df.dropna()
host_id_df = pd.read_json('host_id.json', orient='index')

In [9]:
host_id_df = pd.read_json('host_id.json', orient='index')
host_id_df.reset_index(inplace=True)
host_id_df = host_id_df.rename(columns = {'index':'speaker'})

In [10]:
host_id_df['speaker'] = host_id_df['speaker'].astype('string')
host_id_df['speaker'] = host_id_df['speaker'].str.upper()
host_id_df.rename(columns={0:'host_id'}, inplace=True)

In [11]:
mergedf = df.merge(host_id_df, how='left', on='speaker')

In [12]:
mergedf['host_id'] = mergedf['host_id'].fillna(-1)

In [13]:
PreProcess_df = mergedf[mergedf['host_id'] != -1]

In [14]:
PreProcess_df = PreProcess_df.loc[PreProcess_df['utterance_len'] >= 100]

In [15]:
top_speakers = PreProcess_df.groupby(['speaker']).size().loc[PreProcess_df.groupby(['speaker']).size() > 20000]
df = pd.DataFrame(PreProcess_df.loc[PreProcess_df['speaker'].isin(top_speakers.index.values)])
df = df.reset_index(drop=True)

In [16]:
df['utterance'] = df['utterance'].str.lower()

In [17]:
df['unidentified'] = df['utterance'].str.startswith('unidentified')
df = df[df['unidentified'] == False]

In [18]:
df.count()

episode          289355
episode_order    289355
speaker          289355
utterance        289355
utterance_len    289355
host_id          289355
unidentified     289355
dtype: int64

In [19]:
df['speaker'].value_counts()

NEAL CONAN        126971
STEVE INSKEEP      33787
ROBERT SIEGEL      29948
IRA FLATOW         29888
FARAI CHIDEYA      23937
MELISSA BLOCK      23134
RENEE MONTAGNE     21690
Name: speaker, dtype: int64

In [20]:
df = df.groupby('speaker').apply(lambda x: x.sample(20000))
df['speaker'].value_counts()

NEAL CONAN        20000
MELISSA BLOCK     20000
ROBERT SIEGEL     20000
IRA FLATOW        20000
RENEE MONTAGNE    20000
STEVE INSKEEP     20000
FARAI CHIDEYA     20000
Name: speaker, dtype: int64

In [21]:
def add_label(x):
    if x=='NEAL CONAN':
        return 1
    elif x=='STEVE INSKEEP':
        return 2
    elif x=='ROBERT SIEGEL':
        return 3
    elif x=='IRA FLATOW':
        return 4
    elif x=='FARAI CHIDEYA':
        return 5
    elif x=='MELISSA BLOCK':
        return 6
    else:
        return 7
 
df['label'] = df['speaker'].apply(add_label)
df = df.reset_index(drop=True)
df.head()

,episode,episode_order,speaker,utterance,utterance_len,host_id,unidentified,label
0,30944,1,FARAI CHIDEYA,it's been nearly two years since the state of ...,386.0,29.0,False,5
1,64052,3,FARAI CHIDEYA,joining us from nassau broadcasting in princet...,556.0,29.0,False,5
2,51944,25,FARAI CHIDEYA,"but you know, now that barack obama had this m...",404.0,29.0,False,5
3,4679,13,FARAI CHIDEYA,"it's the - you know - so, that's something tha...",121.0,29.0,False,5
4,51964,1,FARAI CHIDEYA,it's a field where homosexuality is often tabo...,244.0,29.0,False,5


In [22]:
df = df.reset_index(drop=True)
df.head()

,episode,episode_order,speaker,utterance,utterance_len,host_id,unidentified,label
0,30944,1,FARAI CHIDEYA,it's been nearly two years since the state of ...,386.0,29.0,False,5
1,64052,3,FARAI CHIDEYA,joining us from nassau broadcasting in princet...,556.0,29.0,False,5
2,51944,25,FARAI CHIDEYA,"but you know, now that barack obama had this m...",404.0,29.0,False,5
3,4679,13,FARAI CHIDEYA,"it's the - you know - so, that's something tha...",121.0,29.0,False,5
4,51964,1,FARAI CHIDEYA,it's a field where homosexuality is often tabo...,244.0,29.0,False,5


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df['utterance'],df['label'], stratify=df['label'])

In [25]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [26]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [27]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [28]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [29]:
keras_metrics = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=keras_metrics)

In [30]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
3282/3282 [==============================] - 9093s 3s/step - loss: -2137.4939 - accuracy: 0.1427 - precision: 1.0000 - recall: 0.9981
Epoch 2/5
3282/3282 [==============================] - 9224s 3s/step - loss: -6423.1162 - accuracy: 0.1428 - precision: 1.0000 - recall: 0.9986
Epoch 3/5
3282/3282 [==============================] - 9082s 3s/step - loss: -10704.0605 - accuracy: 0.1428 - precision: 1.0000 - recall: 0.9987
Epoch 4/5
3282/3282 [==============================] - 9014s 3s/step - loss: -14997.4346 - accuracy: 0.1428 - precision: 1.0000 - recall: 0.9987
Epoch 5/5
3282/3282 [==============================] - 9055s 3s/step - loss: -19286.7969 - accuracy: 0.1428 - precision: 1.0000 - recall: 0.9987
